In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#CPCG0100

In [3]:
#Read the data

GenomicFeatures_CPCG0100 = pd.read_csv(open('data/CPCG0100/GenomicFeatures_CPCG0100.txt','r'),sep='\t')
SNVCalls_CPCG0100    = pd.read_csv(open('data/CPCG0100/SNVCalls_CPCG0100.txt','r'),sep='\t') 

In [4]:
print GenomicFeatures_CPCG0100.shape
print SNVCalls_CPCG0100.shape

(198526, 15)
(198526, 182)


In [5]:
GenomicFeatures_CPCG0100.head()

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449


In [6]:
SNVCalls_CPCG0100.head()

,CHROM,POS,END,X2399002,X2673208,X2673225,X2677201,X2677205,X2679037,X2679040,X2711267,X2762064,X2762066,X2787011,X2787148,X2787658,X2787662,X2788241,X2788246,X2788251,
0,chr1,10407,10408,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,chr1,10412,10413,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,chr1,16579,16580,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
3,chr1,17518,17519,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,chr1,17537,17538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [7]:
np.unique(GenomicFeatures_CPCG0100['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CNN', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA',
       'GCC', 'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'NCG', 'NTC',
       'TCA', 'TCC', 'TCG', 'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [8]:
CPCG0100 = pd.merge(GenomicFeatures_CPCG0100, SNVCalls_CPCG0100,left_on=['CHROM', 'POS'], right_on=['CHROM','END'])


In [52]:
CPCG0100.head() #182+15

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,END,X2399002,X2673208,X2673225,
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965,10407,10408,0,0,0,...
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179,10412,10413,0,0,0,...
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766,16579,16580,0,0,0,...
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185,17518,17519,0,0,0,...
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449,17537,17538,0,0,0,...


In [10]:
CPCG0100_X = CPCG0100.iloc[:,2:15] #Exclude CHROM, POS
CPCG0100_Y = CPCG0100.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [11]:
print CPCG0100_X.shape
print CPCG0100_Y.shape

(198526, 13)
(198526, 179)


In [12]:
CPCG0100_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179


In [13]:
CPCG0100_Y[:2]

,X2399002,X2673208,X2673225,X2677201,X2677205,X2679037,X2679040,X2711267,X2762064,X2762066,X2787011,X2787148,X2787658,X2787662,X2788241,X2788246,X2788251,X2788254,X2791036,X2791292,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...


In [14]:
totalSample = CPCG0100_Y.shape[0]
CPCG0100_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print CPCG0100_Y_Val.shape

198526
(198526, 1)


In [15]:
CPCG0100_Y.shape

(198526, 179)

In [22]:
CPCG0100_Y.values[0].shape

(96,)

In [16]:
for i in range(totalSample):
    CPCG0100_Y_Val[i] = np.argmax(np.bincount(CPCG0100_Y.values[i]))

In [17]:
CPCG0100_Y_Val_1 = pd.DataFrame(CPCG0100_Y_Val)
CPCG0100_Y_Val_1.columns = ['Label']

In [18]:
CPCG0100_Y_Val_1[:3]

,Label
0,0
1,0
2,0


In [19]:
CPCG0100_dataframe = pd.concat([CPCG0100_X, CPCG0100_Y_Val_1],axis=1)

In [20]:
CPCG0100_dataframe.shape

(198526, 14)

In [48]:
CPCG0100_dataframe.head()

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Label
0,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965,0
1,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179,0
2,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766,0
3,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185,0
4,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449,0


In [21]:
CPCG0100_dataframe.to_csv('CPCG0100.csv')

In [22]:
#Now we have variable importance ...lets do analysis

In [23]:
%load_ext rpy2.ipython

In [25]:
%%R
# Read in the data
CPCG0100Frame = read.csv("CPCG0100.csv")
str(CPCG0100Frame)

'data.frame':	198526 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  183 177 115 83 96 120 117 92 27 9 ...
 $ NonRefAllele   : int  3 10 10 26 20 11 14 8 3 3 ...
 $ BaseQual       : int  4490 4691 3512 3251 3255 3672 3865 2946 524 342 ...
 $ TumourCoverage : int  231 229 137 119 127 155 155 123 36 17 ...
 $ NormalCoverage : int  133 134 74 59 53 108 108 75 26 15 ...
 $ MapQual        : num  25 25 0 37 29 0 0 0 27 0 ...
 $ ReadPosition   : num  55 60 52 38 47 42 51 55 70.5 64 ...
 $ Trinucleotide  : Factor w/ 35 levels "ACA","ACC","ACG",..: 32 14 28 18 19 24 34 18 35 6 ...
 $ HomopolymerRate: num  4.7 4.53 2.71 2.75 2.72 ...
 $ GCcontent      : num  0.587 0.587 0.532 0.677 0.677 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 1 1 8 1 1 8 1 1 5 ...
 $ DistanceSNP    : int  41830 41825 35658 34719 34700 32011 31988 23727 23645 10933 ...
 $ StrandBias     : num  0.368 0.371 0.336 0.462 0.409 ...
 $ Label          : num  0 0

In [26]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
sum(is.na(CPCG0100Frame$ReadPosition)) 

[1] 32


In [27]:
%%R
CPCG0100Frame$ReadPosition[is.na(CPCG0100Frame$ReadPosition)] = mean(CPCG0100Frame$ReadPosition, na.rm=TRUE)
CPCG0100Frame$StrandBias[is.na(CPCG0100Frame$StrandBias)] = mean(CPCG0100Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0100Frame$ReadPosition)) )
print (sum(is.na(CPCG0100Frame$StrandBias)) )

[1] 0
[1] 0


In [28]:
%%R
sum(is.na(CPCG0100Frame))  #No NA value now

[1] 0


In [29]:
%%R
write.csv(CPCG0100Frame, file = "CPCG0100_imputed.csv")  #Cleaned CSV without NA

In [30]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 hours 40 minutes 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.56 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [31]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0100_imputed.csv"
CPCG0100.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0100.hex") 
train                 = as.data.frame(CPCG0100.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [32]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [33]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
        variable relative_importance scaled_importance percentage
1   NonRefAllele            1.000000          1.000000   0.393815
2 NormalCoverage            0.105450          0.105450   0.041528
3      RefAllele            0.083199          0.083199   0.032765
4 TumourCoverage            0.072141          0.072141   0.028410
5       BaseQual            0.059017          0.059017   0.023242

---
                    variable relative_importance scaled_importance percentage
58         Trinucleotide.CCT            0.005477          0.005477   0.002157
59                StrandBias            0.004780          0.004780   0.001882
60         Trinucleotide.TCC            0.004598          0.004598   0.001811
61         Trinucleotide.TCT            0.004549          0.004549   0.001791
62  GenomicLocation.intronic            0.002152          0.002152   0.000848
63 Trinucleotide.missing(NA)            0.000000          0.000000   0.000000


In [34]:
%%R
write.table(var_importance, file="CPCG0100_variable_importance.csv")

In [35]:
var_importance = pd.read_csv(open('CPCG0100_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [36]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.393815
2,NormalCoverage,0.105450,0.105450,0.041528
3,RefAllele,0.083199,0.083199,0.032765
4,TumourCoverage,0.072141,0.072141,0.028410
5,BaseQual,0.059017,0.059017,0.023242


In [37]:
GenomicFeatures_CPCG0100[:5]

,CHROM,POS,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias
0,chr1,10408,183,3,4490,231,133,25,55,TTA,4.696078,0.587065,NaN,41830,0.367965
1,chr1,10413,177,10,4691,229,134,25,60,CTA,4.528846,0.587065,NaN,41825,0.371179
2,chr1,16580,115,10,3512,137,74,0,52,TCA,2.707143,0.532338,NaN,35658,0.335766
3,chr1,17519,83,26,3251,119,59,37,38,GCA,2.746377,0.676617,ncRNA_intronic,34719,0.462185
4,chr1,17538,96,20,3255,127,53,29,47,GCC,2.719424,0.676617,NaN,34700,0.409449


In [38]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.059016659855842597,
 'DistanceSNP': 0.023346446454525004,
 'GCcontent': 0.0057310732081532496,
 'GenomicLocation.UTR3': 0.056907754391431802,
 'GenomicLocation.UTR5': 0.044845979660749401,
 'GenomicLocation.downstream': 0.049555622041225399,
 'GenomicLocation.exonic': 0.0092172222211956995,
 'GenomicLocation.exonic;splicing': 0.0450383238494396,
 'GenomicLocation.intergenic': 0.0057625533081591103,
 'GenomicLocation.intronic': 0.0021524797193706001,
 'GenomicLocation.missing(NA)': 0.0154282161965966,
 'GenomicLocation.ncRNA_UTR3': 0.040674488991498899,
 'GenomicLocation.ncRNA_UTR5': 0.038808260113000898,
 'GenomicLocation.ncRNA_exonic': 0.0303917564451694,
 'GenomicLocation.ncRNA_intronic': 0.0091871432960033399,
 'GenomicLocation.ncRNA_splicing': 0.040368977934122099,
 'GenomicLocation.splicing': 0.034456580877304098,
 'GenomicLocation.upstream': 0.051519073545932798,
 'GenomicLocation.upstream;downstream': 0.019780052825808501,
 'HomopolymerRate': 0.024760439991950999,

In [40]:
noOfSample = GenomicFeatures_CPCG0100.shape[0]

CPCG0100_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_CPCG0100.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_CPCG0100.iloc[i,2]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_CPCG0100.iloc[i,3]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_CPCG0100.iloc[i,4]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_CPCG0100.iloc[i,5]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_CPCG0100.iloc[i,6]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_CPCG0100.iloc[i,7]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_CPCG0100.iloc[i,8]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_CPCG0100.iloc[i,9], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_CPCG0100.iloc[i,9])]
    HomopolymerRate= GenomicFeatures_CPCG0100.iloc[i,10]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_CPCG0100.iloc[i,11]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_CPCG0100.iloc[i,12], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_CPCG0100.iloc[i,12])]
    DistanceSNP    = GenomicFeatures_CPCG0100.iloc[i,13]*var_imp_dict['DistanceSNP']
    StrandBias     = GenomicFeatures_CPCG0100.iloc[i,14]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' DistanceSNP=',DistanceSNP
    #print 'StrandBias=',StrandBias
    CPCG0100_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+DistanceSNP+StrandBias

In [41]:
#Sorting dictionary
CPCG0100_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in CPCG0100_Dict.items()],reverse=True)]

In [42]:
CPCG0100_Dict_Frame = pd.DataFrame(CPCG0100_Dict1, columns=['POS', 'Value'])

In [43]:
CPCG0100_Dict_Frame.to_csv("CPCG0100_Dict_Frame.csv")

In [58]:
GenomicFeatures_CPCG0100_1 = pd.concat([CPCG0100.iloc[:,0:16], CPCG0100_Y_Val_1],axis=1)

In [60]:
rows = CPCG0100_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_CPCG0100_Top20 = GenomicFeatures_CPCG0100_1.loc[GenomicFeatures_CPCG0100_1['POS_x'].isin(rows)]

[ 27732070     63653    185807    623693    640356  61969229    788366
    816818  61969437    829649  61968977  61969180  61969371  61968871
  61969385  61969386 196625733   1243524  61970338   1413967]


In [62]:
GenomicFeatures_CPCG0100_Top20[:10]

,CHROM,POS_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,POS_y,Label
80227,chr19,27732070,6965,1035,249892,357720,322540,17,65,CTT,2.342466,0.383085,NaN,8640,0.540655,27732069,0
122132,chr3,196625733,4257,2399,184550,141372,108156,15,67,CCG,2.486111,0.398010,NaN,97,0.368517,196625732,0
160236,chr7,61968871,6982,899,236654,138975,126878,37,42,ATT,2.798561,0.358209,NaN,16301,0.604871,61968870,0
160237,chr7,61968977,7143,853,244685,153132,142931,60,53,ATG,2.178808,0.402985,NaN,16195,0.521544,61968976,0
160238,chr7,61969180,7436,644,224614,157156,147005,36,44,TTT,2.791367,0.353234,NaN,15992,0.520750,61969179,0
160239,chr7,61969229,7162,829,247015,170379,159252,46,49,CCT,2.545455,0.378109,NaN,15943,0.540501,61969228,0
160240,chr7,61969371,7660,374,240318,148311,140982,60,45,ACT,2.517241,0.417910,NaN,15801,0.499740,61969370,0
160241,chr7,61969385,7025,437,201465,147666,139001,60,43,TTT,2.591549,0.417910,NaN,15787,0.514465,61969384,0
160242,chr7,61969386,7368,174,199742,148185,139119,60,43,TTT,2.559441,0.422886,NaN,15786,0.513736,61969385,0
160243,chr7,61969437,7810,185,251866,163187,154133,60,51,GCA,2.573427,0.398010,NaN,15735,0.475877,61969436,0


In [46]:
GenomicFeatures_CPCG0100_Top20.to_csv('GenomicFeatures_CPCG0100_Top20.csv')

In [50]:
rows = CPCG0100_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_CPCG0100_Top20 = CPCG0100_dataframe.loc[CPCG0100_dataframe['POS'].isin(rows)]

[ 27732070     63653    185807    623693    640356  61969229    788366
    816818  61969437    829649  61968977  61969180  61969371  61968871
  61969385  61969386 196625733   1243524  61970338   1413967]


KeyError: u'no item named POS'